In [ ]:
import google_play_scraper #https://pypi.org/project/google-play-scraper/
import play_scraper #https://pypi.org/project/play-scraper/
import pymongo

In [ ]:
## Search for app id using keyword contained in app and title
def write_app_reviews(name, title, database_name):
    app_list = play_scraper.search(name)
    for i in app_list:
        if i.get('title') == title:
            playstore_app_id = i.get('app_id')
            break
        else:
            continue
    
    ## connect to mongodb server
    ## NB. make sure server is already running
    my_client = pymongo.MongoClient('mongodb://localhost:27017')

    ## create database 
    if 'local' in my_client.list_database_names():
        db = my_client[database_name]
        print('Database {} already exist'.format(database_name))
    else:
        print('Creating database: {} '.format(database_name))
        db = my_client[database_name]
        print('Database {} successfully created'.format(database_name))

    result, _ =  google_play_scraper.reviews(playstore_app_id,
                        lang= None, # defaults to 'en'
                        country='de', # defaults to 'us'
                        sort=google_play_scraper.Sort.NEWEST, # defaults to Sort.MOST_RELEVANT
                        count=10000, # defaults to 100
                        filter_score_with= None # defaults to None(means all score)
                        )

    ## add collection to database
    collection_name = "{}_reviews".format(name)
    db = my_client[database_name]
    my_collection = db[collection_name]

    ## insert reviews into collection
    my_collection.insert_many(result)
    print("{} reviews successfully written to {}".format(name, collection_name))

    my_client.close()
    return my_collection

In [ ]:
## examples: scrape review for xing app
my_collection =write_app_reviews("xing","XING – Your jobs and careers network","local")

In [ ]:
my_client = pymongo.MongoClient('mongodb://localhost:27017/')
db = my_client['local']
db.xing_reviews.drop()